# **Get brief inforation from VK Api**

In [1]:
from datetime import datetime
import requests
from time import sleep
import getpass
import pandas as pd

In [2]:
token = getpass.getpass()

 ·····················································································


# Functions

In [3]:
def vk_get(url):
    res = requests.get(url).json()
    sleep(.34)
    return res

In [4]:
def year_post_count(id):
    offset = 0
    count = 0
    while True:
        url = f'https://api.vk.com/method/wall.get?owner_id={id}&offset={offset}&count=100&v=5.131&access_token={token}'
        res = vk_get(url)
        if res['response']['items'][-1]['date'] > datetime_filter:
            count += len(res['response']['items'])
        else:
            count += len([i for i in res['response']['items'] if i['date'] > datetime_filter])
            break
        offset += 100
    
    return count


In [5]:
def subscriber_count(id):
    url = f'https://api.vk.com/method/groups.getMembers?group_id={id}&v=5.131&access_token={token}'
    res = vk_get(url)
    if 'response' in res:
        res = res['response']['count']
    else:
        res = 'Access denied'
    return res

In [6]:
def friend_count(id):
    url = f'https://api.vk.com/method/friends.get?user_id={id}&v=5.131&access_token={token}'
    res = vk_get(url)
    if 'response' in res:
        res = res['response']['count']
    else:
        res = 'Access denied'
    return res

# Data

In [7]:
datetime_filter = datetime(2022, 1, 1, 0, 0, 0).timestamp()

## Groups

In [8]:
data = [131743773,
        'kuban_gov',
        'krdru',
        'krasnoyarskkr',
        'krasnoyarskrf',
        'gov.tuva', 
        'mkyzyl_100',
        'udmurt_gov',
        'izh_gov',
        'grozmer',
        'tverreg'
        'tverrussia69'
       ]

In [9]:
url = 'https://api.vk.com/method/groups.getById?group_ids={ids}&v=5.131&access_token={token}'.format(
    ids=','.join(str(x) for x in data), 
    token=token)

In [10]:
res = vk_get(url)
df_g = pd.DataFrame([(i['id'], i['name'], i['screen_name']) for i in res['response']], columns=['id', 'name', 'screen_name'])

In [11]:
df_g['subscribers'] = df_g['id'].apply(subscriber_count)
df_g['year_posts'] = df_g['id'].apply(lambda x: year_post_count(x*-1))
df_g['type'] = 'group'

## Users

In [12]:
data = [
    'kondratyevvi',
    'uss_av',
    'vladislavkhovalyg',
    238214072,
    'a.brechalov',
    'olegbekmemetev',
    'ramzan'
]

In [13]:
url = 'https://api.vk.com/method/users.get?user_ids={ids}&fields=domain&v=5.131&access_token={token}'.format(
    ids=','.join(str(x) for x in data), 
    token=token)

In [14]:
res = vk_get(url)
df_u = pd.DataFrame([(i['id'], f"{i['first_name']} {i['last_name']}", i['domain']) for i in res['response']], columns=['id', 'name', 'screen_name'])

In [15]:
df_u['subscribers'] = df_u['id'].apply(friend_count)
df_u['year_posts'] = df_u['id'].apply(year_post_count)
df_u['type'] = 'user'

-----

In [16]:
df = pd.concat([df_g, df_u], axis=0)

In [17]:
df

,id,name,screen_name,subscribers,year_posts,type
0,131743773,Администрация Петропавловска-Камчатского,club131743773,98,368,group
1,192173260,Краснодарский край,kuban_gov,99321,141,group
2,28402455,Город Краснодар,krdru,Access denied,239,group
3,196936695,Правительство Красноярского края,krasnoyarskkr,50715,184,group
4,121269719,Город Красноярск,krasnoyarskrf,38585,236,group
5,34709566,Правительство Республики Тыва,gov.tuva,65215,162,group
6,77016543,Мэрия города Кызыла,mkyzyl_100,51265,130,group
7,192431671,Удмуртская Республика,udmurt_gov,146155,125,group
8,69700574,Администрация Ижевска,izh_gov,10911,272,group
9,119042965,Мэрия г.Грозный,grozmer,713,50,group
